In [294]:
#!/usr/bin/env Python
# coding=utf-8
import re
from PIL import Image
import os

We have two file path.
One is dim_items（new).txt, this is the original one, which includes all the information of both training and test data. We use this data set because we would like to get the key information of the test data.
We store the result in a list result_all

In [295]:
path_all= "dim_items（new).txt"

file_all= open(path_all, encoding="utf-8")

In [296]:
seq_all= re.compile("\s+")
result_all= []
for line in file_all:
    lst = seq_all.split(line.strip())
    item = {
        "item": lst[0],
        "cat": lst[1],
        "terms":lst[2:]
    }
    result_all.append(item)    
file_all.close()
print(result_all[0])

{'item': '29', 'cat': '155', 'terms': ['123950,53517,106068,59598,7503,171811,25618,147905,203432,123580,178091,154365,127004,31897,82406']}


Another one is items_feature.txt, which has already processed by a member of our team, only contained the information of the training set, we use this data set to create wordbegs and do further training.
We store the result in a list result

In [297]:
path = "items_feature.txt"

file = open(path, encoding="utf-8")

In [298]:
seq = re.compile("\s+")
result = []
for line in file:
    lst = seq.split(line.strip())
    item = {
        "item": lst[0],
        "cat": lst[1],
        "terms":lst[2:]
    }
    result.append(item)    
file.close()
print(result[0])

{'item': '59', 'cat': '284', 'terms': ['123950,38910,22837,5026,15459,47776,158346,101881,131272,176333,196079,23211,148988,144893,167633']}


At this time we just find the nearest training sets for our fist test item, and we will find all the results later on, but this time, just the first one as an esample, so don't worry.
The first item in test sample is 1417

In [299]:
# the first test item is 1417:
for i in range(len(result_all)):
    if result_all[i]['item'] == '3967':
        test_item = result_all[i]
        print (test_item)
        break

{'item': '3967', 'cat': '160', 'terms': ['48909,53517,116593,158346,25754,99096,217074,99641,123776,20864,94517,23034,77505,146990,24893']}


In [300]:
test = test_item['terms'][0]
test = test.split(',', -1)
test = [int(i) for i in test]

In [301]:
print (test)

[48909, 53517, 116593, 158346, 25754, 99096, 217074, 99641, 123776, 20864, 94517, 23034, 77505, 146990, 24893]


Now we get the information of test item 1417, its category is 33, with keywords[188391, 211242, 179516, 63471, 179308, 105847, 24893, 81538, 7911, 64409, 118724, 24893, 94517, 201722, 146990, 147211, 168934]
We transfer those keywords into int type and store it in a list, that helps to make our test word vector

Next we will find all the traing samples which has the same category as test sample 1417, we store them in a new list name new_result. This reduces the numbers of datas and will surely promote the training speed 

In [302]:
new_result  =[]
for i in range(len(result)):
    if result[i]['cat'] == test_item['cat']:
        new_result.append(result[i])

In [303]:
print (len(new_result))

1755


Now we are going to create our wordbags.
Here are some useful information about bags of words:
Bag-of-words: http://en.wikipedia.org/wiki/Bag-of-words model

Our model is simply inspired by that, we create a bag, which contains all the keys word in the traing model. We store all the keywords in a directory named key

In [304]:
dic = { }
for i in range(len(new_result)):
    c = new_result[i]['terms'][0]
    c = c.split(',', -1)
    c = [int(i) for i in c]
    new_result[i]['words'] = c
    del(new_result[i]['terms'])
    for each_i in c:
        if each_i in dic:
            continue
        else:
            dic[each_i] = 1
keys = dic.keys()            
#print (keys)
    

In [305]:
key = [i for i in keys]

In [306]:
print (key)

[16235, 17335, 48909, 53517, 129681, 158346, 92493, 190249, 140555, 98969, 146990, 20864, 15463, 24893, 77505, 123950, 33202, 116593, 99641, 217074, 45575, 31897, 204385, 142446, 154365, 191704, 102728, 214169, 142053, 218823, 191821, 81578, 73010, 114138, 87937, 38969, 38910, 192848, 153471, 59598, 140284, 119535, 196505, 38719, 198337, 100053, 152195, 154284, 149240, 22837, 94517, 5026, 121419, 105847, 176333, 170709, 27497, 91395, 123580, 52731, 209681, 56917, 97413, 60006, 82025, 128660, 181179, 102549, 139570, 102692, 64409, 187580, 91747, 63471, 187983, 90102, 189093, 124645, 33145, 50236, 27656, 207049, 189662, 186525, 51664, 205852, 112571, 57524, 161872, 110696, 195084, 1664, 176762, 172394, 133005, 25637, 144638, 166367, 218687, 198206, 103412, 64578, 184435, 129569, 23034, 81538, 128913, 124585, 38803, 125706, 76342, 76476, 111513, 164553, 205714, 15633, 7911, 147893, 61580, 127051, 54253, 79356, 161620, 102291, 80085, 129889, 21288, 71017, 185279, 200395, 131272, 133834, 20

In [307]:
print(len(key))

2238


This just shows that the items in the new_result has the same category as test sample 1417, both are 33

In [308]:
print (new_result[0])

{'item': '894', 'cat': '160', 'words': [16235, 17335, 48909, 53517, 129681, 158346, 92493, 190249, 140555, 98969, 146990, 20864, 15463, 24893, 77505]}


Now we create the word vectors for our training sample. Since the lens of keys are 2389, each vector should has the same length of the len(keys) 

In [309]:
train = [ ]
for i in range(len(new_result)):
    tmp = []
    for j in range(len(key)):
        if key[j] in new_result[i]['words']:
            tmp.append(1)
        else:
            tmp.append(0)
            
    train.append(tmp)

In [310]:
print (len(train))
print (len(train[0]))
print ("Is len(key) == len(train vecote)", len(key) == len(train[0]))

1755
2238
Is len(key) == len(train vecote) True


In [311]:
train_label = [i for i in range(len(new_result))]

Now we create word vector for our test sample 1417, and store it into a to_list.

In [312]:
to_predict = []
for j in range(len(key)):
    if key[j] in test:
        to_predict.append(1)
    else:
        to_predict.append(0)
            

Below are just to check

In [313]:
print (len(train) == len(train_label))
print (len(to_predict) == len(train[0]))
print (len(to_predict))
print (len(train[0]))

True
True
2238
2238


Here we are going to imply some machine learning algorithm to find the training sample which has the most similar word vector with our test sample.
We will imply KNN, and set k to 1, so it will return the nearest one

In [314]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(train, train_label) 


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [315]:
predict = neigh.predict([to_predict])

In [316]:
print (test)

[48909, 53517, 116593, 158346, 25754, 99096, 217074, 99641, 123776, 20864, 94517, 23034, 77505, 146990, 24893]


Now we got the item which has the most similar keywords with our test sample

In [317]:
print (new_result[predict[0]])

{'item': '1403439', 'cat': '160', 'words': [92119, 20864, 24893, 48909, 53517, 146990, 217301, 77505, 158346, 217074, 94517, 45575, 20864, 24893]}


And we plot it to see whether the test and training are similar

In [318]:
for i in range(1, 5):
    file_path = "../tianchi_fm_img2_"+str(i)+"/"+str(test_item['item'])+".jpg"
    if os.path.isfile(file_path):
        IMG = Image.open(file_path)
        IMG.show()
        break

In [319]:
for i in range(1, 5):
    file_path = "../tianchi_fm_img2_"+str(i)+"/"+str(new_result[predict[0]]['item'])+".jpg"
    if os.path.isfile(file_path):
        IMG = Image.open(file_path)
        IMG.show()
        break

In [320]:
path_match= "dim_fashion_matchsets（new).txt"

file_match= open(path_match, encoding="utf-8")

In [321]:
seq_match= re.compile("\s+")
result_match= []
for line in file_match:
    lst = seq_match.split(line.strip())
    item = {
        "id": lst[0],
        "matches": lst[1],
    }
    result_match.append(item['matches'])    
file_match.close()
print(result_match[0])

160870;3118604


In [322]:
print (result_match[1])

1842610;2741506


In [323]:
match = []
for i in range(len(result_match)):
    c = result_match[i]
    c = c.split(';', -1)
    c = [i for i in c]
    match.append(c)

In [324]:
Match_train  = []
for i in range(len(match)):
    tmp = []
    for c in match[i]:
        c = c.split(',', -1)
        c = [int(i) for i in c]
        tmp.append(c)
    Match_train.append(tmp)
        

In [325]:
print (Match_train[2])

[[893028], [993019, 1375599, 1913565, 3036503], [2849440], [2546147], [2329974, 2661094, 347849], [884801, 127779, 3122713], [2338561]]


In [326]:
print (match[2])

['893028', '993019,1375599,1913565,3036503', '2849440', '2546147', '2329974,2661094,347849', '884801,127779,3122713', '2338561']


In [327]:
to_find = int(new_result[predict[0]]['item'])
print (to_find)

1403439


In [328]:
for i in range(len(Match_train)):
    for j in Match_train[i]:
        if to_find in j:
            find = Match_train[i]
            break

In [329]:
print (find)

[[186256], [1403439], [2736153], [3049716], [2310785]]


In [330]:
for j in find:
    if to_find not in j:
        for k in range(1, 5):
            file_path = "../tianchi_fm_img2_"+str(k)+"/"+str(j[0])+".jpg"
            if os.path.isfile(file_path):
                IMG = Image.open(file_path)
                IMG.show()
                break